<a href="https://colab.research.google.com/github/martinj2-dot/CSC386-Wall-Follower/blob/main/CSC386%20Wall%20Follower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Potential Fields

This notebook allows us to manipulate and visualize potential fields which can be used for navigation.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

The following block defines a default, small map and defines some functions which can be used to load a map.

In [ ]:
HIGH = 1e6

class MapParams(object):
    def __init__(self):
        self.origin = []
        self.width = None
        self.height = None
        self.meters_per_cell = None
        self.file_path = "test_map.map"

    def as_string_list(self):
        data = self.origin + [self.width, self.height, self.meters_per_cell]
        return [str(ele) for ele in data]


def read_map(map_data, map_file):
    params = MapParams()
    params.file_path = map_file
    map_data = map_data.split("\n")

    # Read header.
    header = map_data.pop(0).strip().split()
    params.origin = [float(ele) for ele in header[:2]]
    params.width, params.height = int(header[2]), int(header[3])
    params.meters_per_cell = float(header[4])

    data = []
    for line in map_data:
        if len(line) == 0:
            continue

        row = line.strip().split(' ')
        row = [int(ele) for ele in row]

        if len(row) != params.width:
            print("Warning: Row has incorrect length", len(row))
            row += [0] * (params.width - len(row))

        data.append(row)

    data = np.array(data)

    # Make sure the data has the right number of rows.
    if data.shape[0] < params.height:
        print("Warning: Map has incorrect number of rows", data.shape[0])
        zeros = np.zeros((params.height - data.shape[0], params.width), dtype=int)
        data = np.concatenate([data, zeros])

    return params, data

params = MapParams()
params.height, params.width = (6, 6)
params.meters_per_cell = 1
img = [0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0]

img = np.array(img, dtype=bool).reshape((params.height, params.width))

## Upload a map file

To upload a map file, run this cell. Don't run it to use the default small image.

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    if not fn.endswith(".map"):
        print("Please upload a .map file.")
    else:
        # Load the map data.
        params, data = read_map(uploaded[fn].decode("utf-8"), fn)
        img = data > 100

## Some useful functions

The function `goal_distances()` returns a grid with the same shape as the image containing the Euclidean distance to the goal at each cell. This includes the square root!

The functions `dt_euclidean()` and `dt_manhattan()` perform the Manhattan and Euclidean distance transform for a given image respectively.

We will use these functions to create attractive and repulsive potentials in the next section.

In [ ]:
def goal_distances(img, goal):
    h, w = img.shape
    idx = np.stack(np.meshgrid(np.arange(h), np.arange(w)), axis=-1)
    dists = ((idx - np.array(goal))**2).sum(axis=2)
    return np.sqrt(dists)

In [ ]:
def dt_euclidean(img):
    H, W = img.shape
    dt = np.zeros((H, W))

    free = np.stack(np.nonzero(np.bitwise_not(img)), axis=1)
    occ = np.stack(np.nonzero(img), axis=1)

    free_norm = (free * free).sum(1).reshape(-1, 1)
    occ_norm = (occ * occ).sum(1).reshape(1, -1)
    dists = free_norm + occ_norm - 2.0 * free.dot(occ.T)

    dt[free[:, 0], free[:, 1]] = dists.min(axis=1)

    return np.sqrt(dt)

In [ ]:
def dt_manhattan(img):
    H, W = img.shape
    dt = np.zeros((H, W))

    free = np.stack(np.nonzero(np.bitwise_not(img)), axis=1)
    occ = np.stack(np.nonzero(img), axis=1)
    dists = np.abs(np.expand_dims(free, axis=-1) - np.expand_dims(occ.T, axis=0)).sum(axis=1)

    dt[free[:, 0], free[:, 1]] = dists.min(axis=1)

    return dt

## Potential Functions

Change these functions to modify the attractive and repulsive potentials.

In [ ]:
def create_attractive_potential(goal_dists):
    """TODO: Change this function to see the effect of different attractive potentials."""
    att_field = np.zeros(goal_dists.shape)  # Put the attractive field here.
    
    """1. The cone potential."""
    att_field = goal_dists / goal_dists.max()  # Normalizes between 0 and 1.
    
    """2. The bowl potential. (Leave cone potential uncommented)"""
    # att_field = att_field * att_field
    
    return att_field

In [ ]:
def create_repulsive_potential(dt):
    """TODO: Change this function to see the effect of different repulsive potentials.
       Only uncomment one of the options."""
    rep_field = np.zeros(goal_dists.shape)  # Put the repulsive field here.

    """1. Normalized negative distance transform."""
    rep_field = -dt / dt.max()

    """2. Thresholded negative distance transform."""
    # Dividing thresh by meters_per_cell converts the threshold from meters to cells, so it doesn't depend on the size 
    # of the map.
    # thresh = 0.3 / params.meters_per_cell  # Try different thresholds!
    # rep_field = np.where(dt > thresh, thresh, dt)
    # rep_field = rep_field / thresh  # Normalizes between 0 and 1.

    """3. The expotential potential."""
    # sigma = 0.1  # Modify sigma to see the effects of different values.
    # rep_field = np.exp(-dt * sigma)

    return rep_field

## Combining the potentials

The following block combines the potential functions by adding them together. You can change which distance transform is used, the goal location, or add any other features you'd like to the final potential here.

In [ ]:
"""TODO: Try different goals."""
GOAL = (50, 50)

"""TODO: Try changing between the Euclidean and Manhattan distance transforms."""
goal_dists = goal_distances(img, GOAL)
# dt = dt_manhattan(img)
dt = dt_euclidean(img)

# Call the potential field functions.
att_field = create_attractive_potential(goal_dists)
rep_field = create_repulsive_potential(dt)

# Combine potential fields.
"""TODO: Perform any modifications to how the overall potential is calculated here."""
potential = att_field + rep_field

## Visualizing the potentials

This cell graphs the image, distance transform, and goal distances, as well as each of the potentials in 2D.

In [ ]:
def plot_img(data, title):
    plt.title(title + "\n")
    ax = plt.gca()
    cax = ax.matshow(data, cmap=plt.get_cmap("viridis"))
    plt.colorbar(cax, shrink=0.8)
    # plt.xticks([])
    # plt.yticks([])


plt.rcParams['figure.figsize'] = (15.0, 10.0)
plt.figure()

plt.subplot(2, 3, 1)
plot_img(img.astype(int), "Image")

plt.subplot(2, 3, 2)
plot_img(dt, "Distance Transform")

plt.subplot(2, 3, 3)
plot_img(goal_dists, "Goal Distances")

plt.subplot(2, 3, 4)
plot_img(att_field, "Attractive Potential")

plt.subplot(2, 3, 5)
plot_img(rep_field, "Repulsive Potential")

plt.subplot(2, 3, 6)
plot_img(potential, "Potential Field")

plt.tight_layout()

plt.show()

This cell graphs each potential in 3D. To modify the viewing angle, pass in view angles (in degrees) to the plot 3D function. For example, this code modifies the view angle:
```python
  plot3D(potential, "Title", [0, 0])  # View from straight on
  plot3D(potential, "Title", [75, 30])  # View from a higher angle
```
The first value controls the height to view from. The second value controls rotation about the _z_-axis.

In [ ]:
def plot3D(field, title, view=[]):
    h, w = field.shape
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    plt.title(title + "\n")

    # Make data.
    X = np.arange(0, w)
    Y = np.arange(0, h)
    X, Y = np.meshgrid(X, Y)

    # Plot the surface.
    surf = ax.plot_surface(X, Y, field, cmap=cm.viridis,
                           linewidth=0, antialiased=False)
    
    if view:
        ax.view_init(view[0], view[1])

    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.7)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

plot3D(att_field, "Attractive Potential")
plot3D(rep_field, "Repulsive Potential")
plot3D(potential, "Potential Field")

plt.show()